<a href="https://colab.research.google.com/github/Navifra-Denny/01_linear_algegra/blob/main/01.pca/principal_component_analysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

from scipy.optimize import minimize_scalar

In [6]:

def generate_clipped_gaussian(mean, std, min_val, max_val, size):
    data = np.random.normal(mean, std, size)
    # Clip to specified range
    data = np.clip(data, min_val, max_val)
    # Round to 1 decimal place
    return np.round(data, 1)


## SS (Sumo of Squares)

![ss](https://raw.githubusercontent.com/Navifra-Denny/01_linear_algegra/main/01.pca/img.png)


> 모든 데이터에서 원점(평균으로 이동된 점)을 지나는 직선에 수선을 발을 내리면, 원점으로부터 수선의 발까지의 길이를 구할 수 있게 된다. 원점을 지나는 직선의 기울기가 변함에 따라, 이 빨간선들의 길이 또한 변하게 된다. PCA 에서는 이 빨간선들의 길이 제곱들의 합이 최대가 되는 직선을 찾는다. 이것이 SS(Sum of Squares) 이다.


## 요약
* 직선은 평균점을 지나며, 기울기 𝑚 에 따라 정의됨:
$$ y = m(x - \bar{x}) + \bar{y} $$
* 각 데이터 포인트 $(x_i, y_i)$ 에 대해 직선에 내린 **수선의 발**을 구하고,
* 그 **수선 길이의 제곱**을 계산한 후, 전체에 대한 합산:
$$ SS(m) = \sum_{i=1}^{N} d_i^2 $$


### 직선과 점 사이의 거리 계산 공식
* 직선: $y=mx+b$
* 점: $(x_0, y_0)$
* 직선과 점 사이 거리:
$$ d = \frac{\left| mx_0 - y_0 + b \right|}{\sqrt{m^2 + 1}} $$

In [7]:
# ======= 핵심 수정: 원점을 지나는 직선에 대한 SS 계산 =======
def calculate_ss_to_origin_projection(x, y, slope):
    """
    각 점을 원점을 지나는 직선 y = mx 위로 수직 투영한 후,
    원점과 투영점 사이의 거리 제곱을 모두 더한 SS 계산
    """
    # 단위 방향 벡터 (직선 y = mx의 방향벡터 [1, m])
    v = np.array([1, slope])
    v = v / np.linalg.norm(v)

    data = np.column_stack((x, y))
    projections = (data @ v[:, np.newaxis]) * v[np.newaxis, :]  # 투영점 좌표들
    distances = np.linalg.norm(projections, axis=1)
    ss = np.sum(distances**2)/1000
    return ss

In [8]:
# ======= 최적 기울기 계산 (SS 최대가 되는 방향) =======
def find_optimal_slope(x, y):
    result = minimize_scalar(
        lambda m: -calculate_ss_to_origin_projection(x, y, m),  # -SS → 최대화
        bounds=(-10, 10), method='bounded'
    )
    return result.x, -result.fun  # slope, max SS

In [9]:
# ======= 최적 기울기 계산 =======
opt_slope, opt_ss = find_optimal_slope(np.array([]), np.array([]))  # dummy, 나중에 재계산
print(f"optimal slope: {opt_slope}, optimal ss: {opt_ss}")

optimal slope: 9.99999335625205, optimal ss: 0.0


In [10]:
# ======= 시각화 업데이트 함수 =======
def update_line(m):
    global optimal_slope, optimal_ss

    # 최적 기울기 갱신 (데이터 생성 이후 최초 1회 수행)
    if optimal_slope == 0 and optimal_ss == 0:
        optimal_slope, optimal_ss = find_optimal_slope(heights, weights)

    plt.figure(figsize=(8, 6))
    plt.scatter(heights, weights, color='skyblue', alpha=0.7, edgecolors='black', label='Data')

    # 평균 축
    plt.axvline(x=height_avg, color='red', linestyle='--', linewidth=0.5, label=f'Height Mean = {height_avg:.1f} cm')
    plt.axhline(y=weight_avg, color='red', linestyle='--', linewidth=0.5, label=f'Weight Mean = {weight_avg:.1f} kg')

    # 사용자 기울기 직선
    x_vals = np.linspace(height_avg - 50, height_avg + 50, 100)
    y_vals = m * (x_vals - height_avg) + weight_avg
    plt.plot(x_vals, y_vals, color='green', linestyle='-', linewidth=2, label=f'Slope = {m:.1f}')

    # 최적 기울기 직선
    y_opt = optimal_slope * (x_vals - height_avg) + weight_avg
    plt.plot(x_vals, y_opt, color='orange', linestyle='--', linewidth=2, label=f'Optimal Slope = {optimal_slope:.2f}')

    # 현재 SS 계산
    ss_value = calculate_ss_to_origin_projection(heights, weights, m)

    # 타이틀 출력
    plt.title(f"SS (User) = {ss_value:.2f} | SS (Optimal) = {optimal_ss:.2f}", fontsize=12)

    # 시각화 설정
    plt.xlim(height_avg - 40, height_avg + 40)
    plt.ylim(weight_avg - 40, weight_avg + 40)
    plt.xlabel("Height (cm)")
    plt.ylabel("Weight (kg)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [11]:
# ======= 데이터 생성 =======
num_samples = 300
height_mean, height_std = 170, 17
weight_mean, weight_std = 80, 13

heights = generate_clipped_gaussian(height_mean, height_std, 0, 210, num_samples)
weights = generate_clipped_gaussian(weight_mean, weight_std, 40, 150, num_samples)

height_avg = np.mean(heights)
weight_avg = np.mean(weights)

# 다시 최적 기울기 계산
optimal_slope, optimal_ss = find_optimal_slope(heights, weights)

# ======= 슬라이더 UI =======
slope_slider = widgets.FloatSlider(
    value=0.0,
    min=-5.0,
    max=5.0,
    step=0.1,
    description='Slope (m):',
    continuous_update=True
)

widgets.interact(update_line, m=slope_slider);

interactive(children=(FloatSlider(value=0.0, description='Slope (m):', max=5.0, min=-5.0), Output()), _dom_cla…